In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install flash-attn --no-build-isolation

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-q3nav2yi/unsloth_b0bbc4a13547439ba73ca11b2ce1a6db
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-q3nav2yi/unsloth_b0bbc4a13547439ba73ca11b2ce1a6db
  Resolved https://github.com/unslothai/unsloth.git to commit 2ff5dc1a8de1614994a275785b7b64fb4db8cb5d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 65.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187831595 sha256=58853b28a5a926cae14402bfd8d4d93a45ebf8f9e79533f37ab09d0d77a99c05
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash-attn


In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048  # 任意の値を選択可能。RoPEスケーリングは内部で自動的にサポート
dtype = None  # 自動検出の場合はNone。Tesla T4、V100の場合はFloat16、Ampere以降の場合はBfloat16
load_in_4bit = True  # メモリ使用量を削減するために4ビット量子化を使用。Falseも可能

model_name = "Yuichi1218/Tetun-unsloth-llama3.1-instruct-bnb4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name= model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token="hf_...",  # meta-llama/Llama-2-7b-hfのようなゲート付きモデルを使用する場合は、これを使用
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.9 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
FastLanguageModel.for_inference(model)  # ネイティブで2倍高速な推論を有効にします
messages = [
    {"role": "system", "content": "You are good translater for Tetun to English"},
    {"role": "user", "content": "Please translate this sentence to English 'Hau aprende linguajen foun iha eskola.'"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    input_ids, streamer=text_streamer, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id
)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


This sentence appears to be in Tetum, the language of East Timor. Here's a translation:

Hau (I) - I
aprendi (learn) - learn
linguajen (language) - language
foun (good) - good
iha (in) - in
eskola (school) - school

So, the sentence "Hau aprendi linguajen foun iha eskola" translates to "I learned a good language in school" or "I learned a good language at school".

Note: Tetum is a language with a complex grammar and spelling system, so
